### clone代码

In [ ]:
!git clone https://github.com/modelscope/modelscope-agent.git

### 安装特定依赖

In [ ]:
! cd modelscope-agent && pip install -r requirements.txt
! pip install transformers_stream_generator
! pip install invisible-watermark>=2.0

In [1]:
import os
#os.chdir('modelscope-agent/demo')
os.chdir('../demo')

import sys
sys.path.append('../')

### 读取环境变量文件和工具config配置

In [2]:
from dotenv import load_dotenv
import os
from modelscope.utils.config import Config

# 配置环境变量，里面需要配置modelscope token来remote调用AI模型
os.environ['TOOL_CONFIG_FILE'] ='../config/cfg_tool_template.json'
os.environ['MODEL_CONFIG_FILE'] ='../config/cfg_model_template.json'

os.environ['OUTPUT_FILE_DIRECTORY'] = '/tmp'

# ModelScope Token从这里获取: https://modelscope.cn/my/myaccesstoken
print('请输入ModelScope Token，可以从这里获取: https://modelscope.cn/my/myaccesstoken')
os.environ['MODELSCOPE_API_TOKEN'] = '61ec67d2-fb1e-4c40-85ed-da2ac54e3a0d'

print('请输入DashScope Token，可以从这里获取: 1. 注册登录https://dashscope.aliyun.com 2. 打开模型广场选择通义千问开源系列，并申请开通通义千问7B模型 3. 预计半天通过')

os.environ['DASHSCOPE_API_KEY'] = 'uwjIui5vzfMXRGfWzdU5hkPdE0FJTFFW95425EAEDCCB11ED9809620D7200B5B8'

# 读取工具调用CONFIG文件，包括工具的名称和远程请求url
tool_cfg_file = os.getenv('TOOL_CONFIG_FILE')
print(tool_cfg_file)
tool_cfg = Config.from_file(tool_cfg_file)

model_cfg_file = os.getenv('MODEL_CONFIG_FILE')
print(model_cfg_file)
model_cfg = Config.from_file(model_cfg_file)



2023-11-15 15:27:59,394 - modelscope - INFO - PyTorch version 2.1.0+cu118 Found.
2023-11-15 15:27:59,397 - modelscope - INFO - TensorFlow version 2.14.0 Found.
2023-11-15 15:27:59,397 - modelscope - INFO - Loading ast index from /mnt/workspace/.cache/modelscope/ast_indexer
2023-11-15 15:27:59,423 - modelscope - INFO - Loading done! Current index file version is 1.9.5, with md5 30066d141994bb686e9cdefbaa0e387f and a total number of 945 components indexed


请输入ModelScope Token，可以从这里获取: https://modelscope.cn/my/myaccesstoken
请输入DashScope Token，可以从这里获取: 1. 注册登录https://dashscope.aliyun.com 2. 打开模型广场选择通义千问开源系列，并申请开通通义千问7B模型 3. 预计半天通过
../config/cfg_tool_template.json
../config/cfg_model_template.json


In [7]:
from modelscope_agent.agent import AgentExecutor
from modelscope_agent.llm import LLMFactory
from modelscope_agent.tools import CodeInterpreter
from modelscope_agent.prompt.chatglm3_prompt import ChatGLMPromptGenerator

model_name = 'chatglm3-6b'
model_cfg["generate_cfg"] = {"do_sample":True, "max_length":512}
llm = LLMFactory.build_llm(model_name, model_cfg)


prompt_generator = ChatGLMPromptGenerator()
additional_tool_list = {
    CodeInterpreter.name: CodeInterpreter()
}
agent = AgentExecutor(llm, tool_cfg, prompt_generator=prompt_generator, additional_tool_list=additional_tool_list, tool_retrieval=False)
available_tool_list = ['code_interpreter']
agent.set_available_tools(available_tool_list)


2023-11-15 15:33:02.384457: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-15 15:33:02.417668: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-15 15:33:02.417696: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-15 15:33:02.417719: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-15 15:33:02.424317: I tensorflow/core/platform/cpu_feature_g

cfg: Config (path: ../config/cfg_model_template.json): {'modelscope-agent': {'type': 'dashscope', 'model': 'modelscope-agent-llm-v1', 'generate_cfg': {'use_raw_prompt': True, 'top_p': 0.8, 'seed': 123, 'debug': False}}, 'qwen_plus': {'type': 'dashscope', 'model': 'qwen-plus', 'generate_cfg': {'use_raw_prompt': True, 'top_p': 0.8, 'top_k': 10, 'seed': 123, 'debug': False}}, 'custom_llm': {'type': 'custom_llm'}, 'openai': {'type': 'openai', 'model': 'gpt-3.5-turbo'}, 'modelscope-agent-7b': {'type': 'modelscope', 'model_id': 'damo/ModelScope-Agent-7B', 'model_revision': 'v1.0.0', 'use_raw_generation_config': True, 'custom_chat': True}, 'chatglm3-6b': {'type': 'modelscope', 'model_id': 'ZhipuAI/chatglm3-6b', 'model_revision': 'v1.0.2'}, 'generate_cfg': {'do_sample': True, 'max_length': 512}}


2023-11-15 15:33:05,813 - modelscope - INFO - Use user-specified model revision: v1.0.2
Loading checkpoint shards: 100%|██████████| 7/7 [00:08<00:00,  1.27s/it]


### 执行测试

In [8]:
# 重置对话，清空对话历史
agent.reset()

# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run("查看我本地python版本并返回", remote=False, print_info=True)

prompt: <|system|>
Answer the following questions as best you can. You have access to the following tools:
[
    {
        "name": "code_interpreter",
        "description": "Executes code on the user's machine, **in the users local environment**, and returns the output",
        "parameters": [
            {
                "name": "language",
                "description": "The programming language (required parameter to the `execute` function)",
                "required": true
            },
            {
                "name": "code",
                "description": "The code to execute (required)",
                "required": true
            }
        ]
    }
]
<|user|>
查看我本地python版本并返回
<|assistant|>
llm_result: 

|LLM inputs in round 1: <|system|>
Answer the following questions as best you can. You have access to the following tools:
[
    {
        "name": "code_interpreter",
        "description": "Executes code on the user's machine, **in the users local environment**, and r

**************************************************round 1**************************************************

[]

In [ ]:

agent.reset()
# remote=True为调用modelscope api，该服务免费支持QPS较低，建议部署在本地，将remote=False
agent.run("generate an image: a cute dog is running in the city", remote=True)